In [69]:
import tensorflow as tf
import matplotlib
import cv2
import os
import pandas as pd
import numpy as np

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
data_dir = "dataset"
train_dir = os.path.join(data_dir, "Train Images")
test_dir = os.path.join(data_dir, "Test Images")

In [4]:
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))

In [5]:
test_df.head(), train_df.head()

(            Image
 0   image6245.jpg
 1  image10409.jpg
 2   image8692.jpg
 3  image10517.jpg
 4   image2580.jpg,
             Image   Class
 0   image7042.jpg    Food
 1   image3327.jpg    misc
 2  image10335.jpg  Attire
 3   image8019.jpg    Food
 4   image2128.jpg  Attire)

In [6]:
CLASS_NAMES = train_df['Class'].unique()

In [7]:
CLASS_NAMES

array(['Food', 'misc', 'Attire', 'Decorationandsignage'], dtype=object)

In [8]:
IMG_HEIGHT = 80
IMG_WIDTH = 80

In [9]:
# from IPython.core.debugger import set_trace

In [10]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.dtypes.float16)
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    return img

In [11]:
# labelled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [12]:
train_df['Image'] = train_df['Image'].apply(lambda x : os.path.join(train_dir, x))
test_df['Image'] = test_df['Image'].apply(lambda x : os.path.join(test_dir, x))

In [13]:
train_df.head()

,Image,Class
0,dataset/Train Images/image7042.jpg,Food
1,dataset/Train Images/image3327.jpg,misc
2,dataset/Train Images/image10335.jpg,Attire
3,dataset/Train Images/image8019.jpg,Food
4,dataset/Train Images/image2128.jpg,Attire


In [14]:
target = train_df.pop('Class')

In [38]:
new_dataset = tf.data.Dataset.from_tensor_slices((train_df.values, target.values))

In [39]:
for file, targ in new_dataset.take(5):
    print(file, targ)

tf.Tensor([b'dataset/Train Images/image7042.jpg'], shape=(1,), dtype=string) tf.Tensor(b'Food', shape=(), dtype=string)
tf.Tensor([b'dataset/Train Images/image3327.jpg'], shape=(1,), dtype=string) tf.Tensor(b'misc', shape=(), dtype=string)
tf.Tensor([b'dataset/Train Images/image10335.jpg'], shape=(1,), dtype=string) tf.Tensor(b'Attire', shape=(), dtype=string)
tf.Tensor([b'dataset/Train Images/image8019.jpg'], shape=(1,), dtype=string) tf.Tensor(b'Food', shape=(), dtype=string)
tf.Tensor([b'dataset/Train Images/image2128.jpg'], shape=(1,), dtype=string) tf.Tensor(b'Attire', shape=(), dtype=string)


In [40]:
full_ds = new_dataset
val_split = 0.2
DS_SIZE = len(train_df)
val_ds = full_ds.take(int(val_split*DS_SIZE))
train_ds = full_ds.skip(int(val_split*DS_SIZE))

In [41]:
for i in val_ds.take(5):
    print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'dataset/Train Images/image7042.jpg'], dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'Food'>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'dataset/Train Images/image3327.jpg'], dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'misc'>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'dataset/Train Images/image10335.jpg'], dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'Attire'>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'dataset/Train Images/image8019.jpg'], dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'Food'>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'dataset/Train Images/image2128.jpg'], dtype=object)>, <tf.Tensor: shape=(), dtype=string, numpy=b'Attire'>)


In [42]:
def get_label(label):
    return label == CLASS_NAMES

In [43]:
def process_data(filepath, label):
    print(filepath, label)
    label = get_label(label)
    img = tf.io.read_file(filepath[0])
    img = decode_img(img)
    return img, label

In [44]:
labelled_ds = new_dataset.map(process_data, num_parallel_calls=AUTOTUNE)

Tensor("args_0:0", shape=(1,), dtype=string) Tensor("args_1:0", shape=(), dtype=string)


In [45]:
labelled_ds

<ParallelMapDataset shapes: ((80, 80, 3), (4,)), types: (tf.float32, tf.bool)>

In [46]:
train_ds = train_ds.map(process_data, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_data, num_parallel_calls=AUTOTUNE)

Tensor("args_0:0", shape=(1,), dtype=string) Tensor("args_1:0", shape=(), dtype=string)
Tensor("args_0:0", shape=(1,), dtype=string) Tensor("args_1:0", shape=(), dtype=string)


In [47]:
for image, label in train_ds.take(1):
    x = image.numpy
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (80, 80, 3)
Label:  [ True False False False]


In [48]:
def augment(image, label):
    image = tf.image.random_crop(image, size=[80, 80, 3]) # Random crop back to 80x80
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5) # Random Contrast
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, max_delta=0.5)
    image = tf.image.random_jpeg_quality(image, min_jpeg_quality=50, max_jpeg_quality=100)
    image = tf.image.random_saturation(image, lower = 0.5, upper=1)
    return image, label

In [49]:
BATCH_SIZE = 64

In [50]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
    ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.shuffle(shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [51]:
def prepare_for_validation(ds, cache=True, shuffle_buffer_size=1000):
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()
            
    ds = ds.batch(BATCH_SIZE)
    ds.prefetch(buffer_size = AUTOTUNE)
    return ds

In [52]:
train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_validation(val_ds)

In [53]:
# image_batch, label_batch = next(iter(train_ds))

In [54]:
import matplotlib.pyplot as plt
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(CLASS_NAMES[label_batch])
        plt.axis('off')

In [55]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
def cnn_model():
    model = tf.keras.Sequential()
    model.add(Conv2D(64, 3, padding="same", activation="relu", input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, 3, padding="valid", activation="relu"))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(len(CLASS_NAMES), activation="softmax"))
    
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    
    return model

In [56]:
model = cnn_model()


In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 80, 80, 64)        1792      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 38, 38, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 19, 19, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 46208)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               5914752   
_________________________________________________________________
dense_3 (Dense)              (None, 4)                

In [58]:
train_ds

<PrefetchDataset shapes: ((None, 80, 80, 3), (None, 4)), types: (tf.float32, tf.bool)>

In [59]:
# train_steps = (1 - val_split)*len(train_df)

history = model.fit(train_ds, epochs=3, steps_per_epoch=4 * len(train_df)//BATCH_SIZE, validation_data = val_ds )

Train for 373 steps, validate for 19 steps
Epoch 1/3
373/373 [==============================] - 847s 2s/step - loss: 1.1158 - accuracy: 0.5230 - val_loss: 1.1611 - val_accuracy: 0.4950
Epoch 2/3
373/373 [==============================] - 851s 2s/step - loss: 0.5487 - accuracy: 0.7896 - val_loss: 1.7761 - val_accuracy: 0.5025
Epoch 3/3
373/373 [==============================] - 850s 2s/step - loss: 0.1360 - accuracy: 0.9613 - val_loss: 2.8514 - val_accuracy: 0.4774


In [60]:
def inceptionV3_model():
    model = tf.keras.applications.InceptionV3(include_top = False, input_shape = (IMG_HEIGHT, IMG_WIDTH, 3), weights = "imagenet")
    model.trainable = False
    x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
    x = tf.keras.layers.Dense(64, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(len(CLASS_NAMES), activation = "softmax")(x)
    
    model = tf.keras.Model(inputs = model.input, outputs = x)
    
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )
    
    return model

In [61]:
transfer_model = inceptionV3_model()

In [62]:
history = transfer_model.fit(train_ds, epochs=3, steps_per_epoch=4*len(train_df)//BATCH_SIZE, validation_data = val_ds)

Train for 373 steps, validate for 19 steps
Epoch 1/3
373/373 [==============================] - 289s 774ms/step - loss: 1.2584 - accuracy: 0.4141 - val_loss: 1.2598 - val_accuracy: 0.4649
Epoch 2/3
373/373 [==============================] - 276s 739ms/step - loss: 1.1732 - accuracy: 0.4703 - val_loss: 1.3273 - val_accuracy: 0.4883
Epoch 3/3
373/373 [==============================] - 276s 741ms/step - loss: 1.1426 - accuracy: 0.4896 - val_loss: 1.4101 - val_accuracy: 0.5067


In [90]:
transfer_model.trainable = True

In [91]:
history_2 = transfer_model.fit(train_ds, epochs=5, steps_per_epoch=len(train_df)//BATCH_SIZE, validation_data = val_ds)

Train for 93 steps, validate for 19 steps
Epoch 1/5
93/93 [==============================] - 114s 1s/step - loss: 1.2104 - accuracy: 0.4474 - val_loss: 1.2826 - val_accuracy: 0.4523
Epoch 2/5
93/93 [==============================] - 114s 1s/step - loss: 1.1817 - accuracy: 0.4627 - val_loss: 1.2808 - val_accuracy: 0.4707
Epoch 3/5
93/93 [==============================] - 114s 1s/step - loss: 1.1787 - accuracy: 0.4691 - val_loss: 1.3261 - val_accuracy: 0.4724
Epoch 4/5
93/93 [==============================] - 115s 1s/step - loss: 1.1584 - accuracy: 0.4800 - val_loss: 1.3449 - val_accuracy: 0.4615
Epoch 5/5
93/93 [==============================] - 110s 1s/step - loss: 1.1585 - accuracy: 0.4829 - val_loss: 1.3668 - val_accuracy: 0.4699


In [63]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_df.values))

In [64]:
def process_data_for_test(filepath):
    img = tf.io.read_file(filepath[0])
    img = decode_img(img)
    return img

In [65]:
test_ds = test_dataset.map(process_data_for_test, num_parallel_calls=AUTOTUNE)

In [66]:
test_ds = prepare_for_validation(test_ds)

In [67]:
pred = transfer_model.predict(test_ds)

In [68]:
pred

array([[2.4688815e-01, 1.1458760e-01, 6.3845152e-01, 7.2732735e-05],
       [7.3968542e-01, 7.0659637e-02, 1.8956001e-01, 9.4929659e-05],
       [8.3930588e-01, 1.5247506e-01, 3.7484064e-03, 4.4707172e-03],
       ...,
       [2.4605204e-01, 6.1186072e-03, 7.4782842e-01, 9.7067050e-07],
       [2.4671352e-01, 4.3973997e-01, 1.9563870e-01, 1.1790786e-01],
       [5.8851486e-01, 2.1835200e-01, 1.9310707e-01, 2.6017169e-05]],
      dtype=float32)

In [73]:
pred_idx = np.argmax(pred, axis = 1)
pred_class_name = [CLASS_NAMES[i] for i in pred_idx]

In [74]:
pred_class_name

['Attire',
 'Food',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Attire',
 'Attire',
 'Food',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'misc',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'Food',
 'Attire',
 'misc',
 'Attire',
 'Food',
 'Food',
 'misc',
 'Decorationandsignage',
 'Food',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Decorationandsignage',
 'Food',
 'Food',
 'Attire',
 'Food',
 'Food',
 'Attire',
 'Attire',
 'Food',


In [75]:
test_df.head()

,Image
0,dataset/Test Images/image6245.jpg
1,dataset/Test Images/image10409.jpg
2,dataset/Test Images/image8692.jpg
3,dataset/Test Images/image10517.jpg
4,dataset/Test Images/image2580.jpg


In [76]:
pred_df = pd.DataFrame(test_df)
pred_df['Class'] = pred_class_name

In [78]:
pred_df['Image'] = pred_df['Image'].apply(lambda x: x.rpartition('/')[-1])

In [79]:
pred_df.head()

,Image,Class
0,image6245.jpg,Attire
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Food


In [81]:
pred_df.to_csv("/mnt/c/Users/Naitik Dodia/Desktop/submission.csv", index=False)